In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

In [ ]:
#do train, validation and test split 
df = pd.read_csv('./features_30_sec.csv')
df['label'] = -1
for i in range(0, len(df)):
    if df.loc[df.index[i], 'filename'][0:5] == 'blues':
      df.loc[df.index[i], 'label'] = 0
    elif df.loc[df.index[i], 'filename'][0:5] == 'class':
      df.loc[df.index[i], 'label'] = 1
    elif df.loc[df.index[i], 'filename'][0:5] == 'count':
      df.loc[df.index[i], 'label'] = 2
    elif df.loc[df.index[i], 'filename'][0:5] == 'disco':
      df.loc[df.index[i], 'label'] = 3
    elif df.loc[df.index[i], 'filename'][0:5] == 'hipho':
      df.loc[df.index[i], 'label'] = 4
    elif df.loc[df.index[i], 'filename'][0:4] == 'jazz':
      df.loc[df.index[i], 'label'] = 5
    elif df.loc[df.index[i], 'filename'][0:5] == 'metal':
      df.loc[df.index[i], 'label'] = 6
    elif df.loc[df.index[i], 'filename'][0:3] == 'pop':
      df.loc[df.index[i], 'label'] = 7
    elif df.loc[df.index[i], 'filename'][0:5] == 'regga':
      df.loc[df.index[i], 'label'] = 8
    elif df.loc[df.index[i], 'filename'][0:4] == 'rock':
      df.loc[df.index[i], 'label'] = 9
indexes = df.index.values.copy()
np.random.seed(0)
np.random.shuffle(indexes)
proportion = np.floor(len(indexes) * 0.2).astype('int')
test_inds = indexes[-proportion:]
test = df.iloc[test_inds, :].copy().reset_index(drop=True)
df=df.reset_index()
valid_inds = indexes[-(2*proportion):-proportion]
valid = df.loc[df['index'].isin(valid_inds)].copy().reset_index(drop=True)
train = df.loc[-(df['index'].isin(test_inds)) & -(df['index'].isin(valid_inds)), :].reset_index(drop=True)

In [ ]:
#get mean and standard deviation for each feature in the training data. This will be used to standardise the data
means = []
stdev = []
for i in range(3, len(train.columns.values)-1):
  means.append(np.mean(train.iloc[:,i]))
  stdev.append(np.std(train.iloc[:,i]))

In [ ]:
#create class dataset
class Dataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X.index)
    
    def __getitem__(self, index):
        image = self.X.iloc[index, ].values.astype(np.uint8)
        image = (image - means )/ stdev #standardise
        if self.y is not None:
            return torch.FloatTensor(image), self.y.iloc[index]
        else:
            return image

In [ ]:
#initialise dataloaders 
train_dataset = Dataset(X=train.iloc[:,3:-1], y=train['label'])
valid_dataset = Dataset(X=valid.iloc[:,3:-1], y=valid['label'])
test_dataset = Dataset(X=test.iloc[:,2:-1])

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [ ]:
#define models
class MLP1(nn.Module):
    def __init__(self):
        super(MLP1, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(57, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
        )
        
    def forward(self, x):
        x = self.layers(x)
        x = F.log_softmax(x, dim=-1)
        return x

class MLP2(nn.Module):
    def __init__(self):
        super(MLP2, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(57, 100),
            nn.ReLU(),
            nn.Linear(100, 10))
        
    def forward(self, x):
        x = self.layers(x)
        x = F.log_softmax(x, dim=-1)
        return x


class MLP3(nn.Module):
    def __init__(self):
        super(MLP3, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(57, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.layers(x)
        x = F.log_softmax(x, dim=-1)
        return x
        


In [ ]:
def training(model, train_loader,valid_loader, optimizer, prefix):
  #Create directories 'models' and 'logs'
  if not os.path.exists('./models/'):
      os.makedirs('./models/')
  if not os.path.exists('./logs/'):
      os.makedirs('./logs/')
      
  train_losses = []
  valid_losses = []
  epochs = 300
  best_val_acc = 0

  

  for epoch in range(epochs):
      model.train()
      correct_tr = 0
      total_tr = 0
      loss_sum=0
      #loop through training data
      for i, (images, labels) in enumerate(train_loader):
          optimizer.zero_grad()
          
          #forward
          outputs = model(images)
          loss = F.nll_loss(outputs, labels) #loss
          loss.backward()
          optimizer.step()
          
          loss_sum+=loss.item()

          #calculate accuracy
          _, predicted = torch.max(outputs.data, 1)
          correct_tr += (predicted == labels).sum().item()
          total_tr += labels.size(0)
        
      train_losses.append(loss_sum)
      valid_loss_sum = 0
      tr_acc = 100*correct_tr/total_tr
      model.eval()
      correct = 0
      total = 0
      #assess on validation data
      with torch.no_grad():
          for i, (images, labels) in enumerate(valid_loader):
              outputs = model(images) #forward
              loss = F.nll_loss(outputs, labels) #loss
              
              valid_loss_sum+= loss.item()
              
              #calculate accuracy
              _, predicted = torch.max(outputs.data, 1)
              correct += (predicted == labels).sum().item()
              total += labels.size(0)
              
      
      valid_losses.append(valid_loss_sum)
      accuracy = 100*correct/total

      #if this is the best accuracy so far, delete other models with the same prefix and save this one
      if accuracy > best_val_acc:
        best_val_acc = accuracy
        for f in os.listdir('./models/'):
            to_remove = '_' + prefix + '_'
            if (to_remove in f)  :
                os.remove(f'./models/{f}')
        torch.save(model, './models/MLP_model_{}_epoch{}_valid acc : {:.2f}%'.format(prefix,epoch, accuracy))

      #if the epoch is divisible by 50, save the losses
      if epoch % 50 == 0:
        pd.DataFrame(train_losses).to_csv('./logs/losses_train_model_{}_epoch_{}.csv'.format(prefix, epoch))
        pd.DataFrame(valid_losses).to_csv('./logs/losses_valid_model_{}_epoch_{}.csv'.format(prefix, epoch))
      
      print('epoch : {}, , train acc : {:.2f}%, total train loss : {:.4f}, total valid loss : {:.4f}, valid acc : {:.2f}%'\
          .format(epoch+1, tr_acc, loss_sum, valid_loss_sum, accuracy))  

In [ ]:
#initialise model and optimiser
model = MLP1()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
training(model, train_loader,valid_loader, optimizer, 'exp1')

epoch : 1, , train acc : 15.50%, total train loss : 247.4943, total valid loss : 68.5314, valid acc : 15.50%
epoch : 2, , train acc : 21.33%, total train loss : 169.5509, total valid loss : 57.4935, valid acc : 25.00%
epoch : 3, , train acc : 29.83%, total train loss : 149.4044, total valid loss : 53.2898, valid acc : 26.50%
epoch : 4, , train acc : 37.50%, total train loss : 136.7678, total valid loss : 50.9579, valid acc : 33.00%
epoch : 5, , train acc : 36.33%, total train loss : 130.2357, total valid loss : 50.0193, valid acc : 32.00%
epoch : 6, , train acc : 41.50%, total train loss : 123.0962, total valid loss : 49.5830, valid acc : 31.50%
epoch : 7, , train acc : 44.83%, total train loss : 118.3873, total valid loss : 49.6484, valid acc : 32.50%
epoch : 8, , train acc : 44.83%, total train loss : 114.7011, total valid loss : 49.0332, valid acc : 35.00%
epoch : 9, , train acc : 46.00%, total train loss : 109.7496, total valid loss : 47.7549, valid acc : 36.00%
epoch : 10, , train

In [ ]:
#change architecture
train_dataset = Dataset(X=train.iloc[:,3:-1], y=train['label'])
valid_dataset = Dataset(X=valid.iloc[:,3:-1], y=valid['label'])
test_dataset = Dataset(X=test.iloc[:,2:-1])

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

model = MLP2()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

training(model, train_loader,valid_loader, optimizer, 'exp2')

epoch : 1, , train acc : 12.00%, total train loss : 71.6827, total valid loss : 18.7562, valid acc : 18.00%
epoch : 2, , train acc : 22.83%, total train loss : 45.0243, total valid loss : 15.7549, valid acc : 25.00%
epoch : 3, , train acc : 31.67%, total train loss : 38.3201, total valid loss : 14.0748, valid acc : 31.50%
epoch : 4, , train acc : 36.17%, total train loss : 34.2220, total valid loss : 13.5859, valid acc : 32.00%
epoch : 5, , train acc : 37.67%, total train loss : 32.7147, total valid loss : 14.7761, valid acc : 32.00%
epoch : 6, , train acc : 38.83%, total train loss : 31.1059, total valid loss : 14.2498, valid acc : 32.00%
epoch : 7, , train acc : 44.83%, total train loss : 29.1304, total valid loss : 13.4457, valid acc : 35.50%
epoch : 8, , train acc : 48.83%, total train loss : 27.3968, total valid loss : 12.7573, valid acc : 37.00%
epoch : 9, , train acc : 49.67%, total train loss : 26.8151, total valid loss : 14.4860, valid acc : 36.50%
epoch : 10, , train acc : 50

In [ ]:
#change architecture
train_dataset = Dataset(X=train.iloc[:,3:-1], y=train['label'])
valid_dataset = Dataset(X=valid.iloc[:,3:-1], y=valid['label'])
test_dataset = Dataset(X=test.iloc[:,2:-1])

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

model = MLP3()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

training(model, train_loader,valid_loader, optimizer, 'exp3')

epoch : 1, , train acc : 16.67%, total train loss : 49.3398, total valid loss : 15.3948, valid acc : 22.00%
epoch : 2, , train acc : 31.50%, total train loss : 37.5799, total valid loss : 14.0807, valid acc : 26.00%
epoch : 3, , train acc : 38.67%, total train loss : 33.9722, total valid loss : 13.7285, valid acc : 33.00%
epoch : 4, , train acc : 41.83%, total train loss : 31.3702, total valid loss : 13.8408, valid acc : 32.00%
epoch : 5, , train acc : 44.50%, total train loss : 30.2132, total valid loss : 12.8452, valid acc : 32.50%
epoch : 6, , train acc : 47.83%, total train loss : 28.3676, total valid loss : 14.0539, valid acc : 35.00%
epoch : 7, , train acc : 48.67%, total train loss : 27.0459, total valid loss : 13.4072, valid acc : 37.00%
epoch : 8, , train acc : 52.17%, total train loss : 25.9309, total valid loss : 14.5279, valid acc : 35.00%
epoch : 9, , train acc : 51.67%, total train loss : 25.2006, total valid loss : 14.2022, valid acc : 37.50%
epoch : 10, , train acc : 55

In [ ]:
#change batch size 
train_dataset = Dataset(X=train.iloc[:,3:-1], y=train['label'])
valid_dataset = Dataset(X=valid.iloc[:,3:-1], y=valid['label'])
test_dataset = Dataset(X=test.iloc[:,2:-1])

train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False)

model = MLP1()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

training(model, train_loader,valid_loader, optimizer, 'exp4')

epoch : 1, , train acc : 12.33%, total train loss : 267.4025, total valid loss : 65.2239, valid acc : 17.50%
epoch : 2, , train acc : 22.50%, total train loss : 168.2177, total valid loss : 57.3435, valid acc : 26.00%
epoch : 3, , train acc : 27.67%, total train loss : 150.4831, total valid loss : 53.8634, valid acc : 24.00%
epoch : 4, , train acc : 34.67%, total train loss : 139.2232, total valid loss : 52.3610, valid acc : 26.00%
epoch : 5, , train acc : 35.33%, total train loss : 132.5792, total valid loss : 50.5504, valid acc : 29.50%
epoch : 6, , train acc : 39.33%, total train loss : 125.7182, total valid loss : 49.6853, valid acc : 30.50%
epoch : 7, , train acc : 41.33%, total train loss : 119.2599, total valid loss : 49.9329, valid acc : 31.00%
epoch : 8, , train acc : 45.00%, total train loss : 115.5173, total valid loss : 49.8839, valid acc : 30.50%
epoch : 9, , train acc : 45.83%, total train loss : 111.6917, total valid loss : 48.3330, valid acc : 35.00%
epoch : 10, , train

In [ ]:
#change model TO MP2
train_dataset = Dataset(X=train.iloc[:,3:-1], y=train['label'])
valid_dataset = Dataset(X=valid.iloc[:,3:-1], y=valid['label'])
test_dataset = Dataset(X=test.iloc[:,2:-1])

train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False)

model = MLP2()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

training(model, train_loader,valid_loader, optimizer, 'exp5')

epoch : 1, , train acc : 16.00%, total train loss : 224.6386, total valid loss : 56.5505, valid acc : 20.00%
epoch : 2, , train acc : 31.33%, total train loss : 144.4000, total valid loss : 48.8052, valid acc : 35.50%
epoch : 3, , train acc : 41.17%, total train loss : 129.7842, total valid loss : 47.7692, valid acc : 35.00%
epoch : 4, , train acc : 42.50%, total train loss : 118.5481, total valid loss : 48.4088, valid acc : 34.50%
epoch : 5, , train acc : 47.33%, total train loss : 110.5936, total valid loss : 47.8713, valid acc : 39.00%
epoch : 6, , train acc : 49.00%, total train loss : 102.4902, total valid loss : 48.0655, valid acc : 38.50%
epoch : 7, , train acc : 56.33%, total train loss : 94.9315, total valid loss : 47.7939, valid acc : 39.50%
epoch : 8, , train acc : 56.17%, total train loss : 89.7409, total valid loss : 47.8106, valid acc : 37.00%
epoch : 9, , train acc : 59.33%, total train loss : 84.5992, total valid loss : 47.8586, valid acc : 40.00%
epoch : 10, , train ac

In [ ]:
#change model to MLP3
train_dataset = Dataset(X=train.iloc[:,3:-1], y=train['label'])
valid_dataset = Dataset(X=valid.iloc[:,3:-1], y=valid['label'])
test_dataset = Dataset(X=test.iloc[:,2:-1])

train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False)

model = MLP3()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

training(model, train_loader,valid_loader, optimizer, 'exp6')

epoch : 1, , train acc : 21.50%, total train loss : 169.0250, total valid loss : 51.3196, valid acc : 28.00%
epoch : 2, , train acc : 33.00%, total train loss : 142.1609, total valid loss : 50.2094, valid acc : 28.00%
epoch : 3, , train acc : 35.33%, total train loss : 130.8836, total valid loss : 48.2623, valid acc : 32.00%
epoch : 4, , train acc : 43.50%, total train loss : 116.4206, total valid loss : 48.8941, valid acc : 32.50%
epoch : 5, , train acc : 47.33%, total train loss : 107.9100, total valid loss : 48.4913, valid acc : 36.00%
epoch : 6, , train acc : 48.50%, total train loss : 103.4640, total valid loss : 48.2293, valid acc : 35.00%
epoch : 7, , train acc : 55.17%, total train loss : 92.6893, total valid loss : 49.1166, valid acc : 34.00%
epoch : 8, , train acc : 58.17%, total train loss : 85.9893, total valid loss : 52.0782, valid acc : 35.00%
epoch : 9, , train acc : 60.33%, total train loss : 80.5513, total valid loss : 50.3226, valid acc : 38.50%
epoch : 10, , train ac

In [ ]:
#change learning rate 
train_dataset = Dataset(X=train.iloc[:,3:-1], y=train['label'])
valid_dataset = Dataset(X=valid.iloc[:,3:-1], y=valid['label'])
test_dataset = Dataset(X=test.iloc[:,2:-1])

train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=8, shuffle=False)

model = MLP1()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

training(model, train_loader,valid_loader, optimizer, 'exp7')

epoch : 1, , train acc : 8.67%, total train loss : 437.1883, total valid loss : 116.1870, valid acc : 10.50%
epoch : 2, , train acc : 13.00%, total train loss : 324.7042, total valid loss : 92.8135, valid acc : 9.50%
epoch : 3, , train acc : 12.67%, total train loss : 268.4388, total valid loss : 80.6732, valid acc : 10.50%
epoch : 4, , train acc : 13.67%, total train loss : 237.4657, total valid loss : 73.9179, valid acc : 9.50%
epoch : 5, , train acc : 14.67%, total train loss : 218.1560, total valid loss : 69.4136, valid acc : 9.50%
epoch : 6, , train acc : 16.83%, total train loss : 205.2861, total valid loss : 66.5151, valid acc : 11.50%
epoch : 7, , train acc : 19.83%, total train loss : 195.7262, total valid loss : 64.3294, valid acc : 16.00%
epoch : 8, , train acc : 21.67%, total train loss : 188.3124, total valid loss : 62.9631, valid acc : 15.50%
epoch : 9, , train acc : 22.67%, total train loss : 182.5330, total valid loss : 61.5472, valid acc : 19.00%
epoch : 10, , train ac